In [1]:
# 판다스 라이브러리
import pandas as pd

In [11]:
# 혜화동 식당 데이터 불러오기
hyehwa = pd.read_csv('Hyehwa.csv')

In [12]:
# 데이터 확인
hyehwa.head()

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
0,MA010120220805454448,낫컴플리트,NaN,I2,음식,I212,비알코올,I21201,카페,I56221,...,1.111020e+24,NaN,서울특별시 종로구 성균관로 12-5,110530,3074,NaN,2,NaN,126.998096,37.584204
1,MA010120220805466468,유환푸드,NaN,I2,음식,I201,한식,I20105,국수/칼국수,I56112,...,1.111020e+24,NaN,서울특별시 종로구 창경궁로35길 13,110530,3076,NaN,1,NaN,127.001484,37.586557
2,MA010120220805488786,버거파크,NaN,I2,음식,I210,기타 간이,I21004,버거,I56111,...,1.111020e+24,NaN,서울특별시 종로구 대학로11길 5,110530,3079,NaN,NaN,NaN,127.001350,37.582159
3,MA010120220805501127,디오엠,NaN,I2,음식,I211,주점,I21104,요리 주점,I56219,...,1.111020e+24,NaN,서울특별시 종로구 성균관로 4-8,110530,3074,NaN,1,NaN,126.998727,37.583615
4,MA010120220805505906,명륜진사갈비,NaN,I2,음식,I201,한식,I20107,돼지고기 구이/찜,I56113,...,1.111020e+24,성균관대학교,서울특별시 종로구 성균관로 25-1,110745,3063,NaN,1,NaN,126.995108,37.585827


In [13]:
# 필요한 컬럼만 정리
hyehwa = hyehwa[['상호명', '행정동명', '위도', '경도']]

In [14]:
# 컬럼명 단순화
hyehwa.columns = ['name',  # 상호명
                  'dong',  # 행정동명
                  'lon',  # 위도
                  'lat'  # 경도
                  ]

# URL 생성

In [6]:
# 필요한 라이브러리 임포트
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import numpy as np
import requests
from bs4 import BeautifulSoup

In [3]:
# 크롬 웹 드라이버 설정 

chromedriver = r'C:/Users/Admin/OneDrive/바탕 화면/Sungkyun University/YeomWon/Python/Driver/chromedriver-win64/chromedriver' 
# 각자 크롬드라이버를 설치한 장소 
# r'주소'하면 절대주소가 됨
driver = webdriver.Chrome(chromedriver) 

C:\Users\Admin\AppData\Local\Temp/ipykernel_2604/3591893826.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)


In [17]:
# [~동 @@식당]으로 네이버 검색어 설정

hyehwa['naver_keyword'] = hyehwa['dong'] + " " + hyehwa['name'] 
hyehwa['naver_map_url'] = ''
hyehwa.head()

,name,dong,lon,lat,naver_keyword,naver_map_url
0,낫컴플리트,혜화동,37.584204,126.998096,혜화동 낫컴플리트,
1,유환푸드,혜화동,37.586557,127.001484,혜화동 유환푸드,
2,버거파크,혜화동,37.582159,127.001350,혜화동 버거파크,
3,디오엠,혜화동,37.583615,126.998727,혜화동 디오엠,
4,명륜진사갈비,혜화동,37.585827,126.995108,혜화동 명륜진사갈비,


In [18]:
# 가게 상세페이지의 URL 수집

for i, keyword in enumerate(hyehwa['naver_keyword'].tolist()):
    print("이번에 찾을 키워드 :", i, f"/ {hyehwa.shape[0] -1} 행", keyword)
    try:
        naver_map_search_url = f"https://m.map.naver.com/search2/search.naver?query={keyword}&sm=hty&style=v5#/list"
        
        driver.get(naver_map_search_url)
        time.sleep(3.5)
        hyehwa.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
        # 네이버 지도 시스템은 data-cid에 url 파라미터를 저장.
        # data-cid 번호를 뽑아두었다가 기본 url 템플릿에 넣어 최종적인 url을 완성.
        
        #검색 결과가 없다면
    except Exception as e1:
        if "li:nth-child(1)" in str(e1):  # -> "child(1)이 없다
            try:
                hyehwa.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
                time.sleep(1)
            except Exception as e2:
                print(e2)
                hyehwa.iloc[i,-1] = np.nan
                time.sleep(1)
        else:
            pass


driver.quit()

이번에 찾을 키워드 : 0 / 607 행 혜화동 낫컴플리트


C:\Users\Admin\AppData\Local\Temp/ipykernel_17032/1736269362.py:10: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  hyehwa.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')


이번에 찾을 키워드 : 1 / 607 행 혜화동 유환푸드


C:\Users\Admin\AppData\Local\Temp/ipykernel_17032/1736269362.py:18: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  hyehwa.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')


Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=117.0.5938.92)
Stacktrace:
	GetHandleVerifier [0x00007FF636897892+54818]
	(No symbol) [0x00007FF636806AC2]
	(No symbol) [0x00007FF6366BDA3B]
	(No symbol) [0x00007FF6366FE4FC]
	(No symbol) [0x00007FF6366FE67C]
	(No symbol) [0x00007FF636739627]
	(No symbol) [0x00007FF63671EAEF]
	(No symbol) [0x00007FF6367375A2]
	(No symbol) [0x00007FF63671E883]
	(No symbol) [0x00007FF6366F3691]
	(No symbol) [0x00007FF6366F48D4]
	GetHandleVerifier [0x00007FF636BFB992+3610402]
	GetHandleVerifier [0x00007FF636C51860+3962352]
	GetHandleVerifier [0x00007FF636C49D4F+3930847]
	GetHandleVerifier [0x00007FF636933646+693206]
	(No symbol) [0x00007FF636811628]
	(No symbol) [0x00007FF63680D934]
	(No symbol) [0x00007FF63680DA62]
	(No symbol) [0x00007FF6367FE113]
	BaseThreadInitThunk [

...

이번에 찾을 키워드 : 453 / 607 행 혜화동 명륜손칼국수
이번에 찾을 키워드 : 454 / 607 행 혜화동 꼬치오뎅
이번에 찾을 키워드 : 455 / 607 행 혜화동 명륜쭈꾸미
이번에 찾을 키워드 : 456 / 607 행 혜화동 김가네김밥
이번에 찾을 키워드 : 457 / 607 행 혜화동 청산만두
이번에 찾을 키워드 : 458 / 607 행 혜화동 소구미
이번에 찾을 키워드 : 459 / 607 행 혜화동 금문
이번에 찾을 키워드 : 460 / 607 행 혜화동 밀가
이번에 찾을 키워드 : 461 / 607 행 혜화동 마산해물탕
이번에 찾을 키워드 : 462 / 607 행 혜화동 진보식당
이번에 찾을 키워드 : 463 / 607 행 혜화동 헤이커피
이번에 찾을 키워드 : 464 / 607 행 혜화동 삼원
이번에 찾을 키워드 : 465 / 607 행 혜화동 구이
이번에 찾을 키워드 : 466 / 607 행 혜화동 일송칼국수
이번에 찾을 키워드 : 467 / 607 행 혜화동 정이가네
이번에 찾을 키워드 : 468 / 607 행 혜화동 692고기포차
이번에 찾을 키워드 : 469 / 607 행 혜화동 후레쉬푸드
이번에 찾을 키워드 : 470 / 607 행 혜화동 한입가득
이번에 찾을 키워드 : 471 / 607 행 혜화동 유니동화
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=117.0.5938.92)
Stacktrace:
	GetHandleVerifier [0x00007FF636897892+54818]
	(No symbol) [0x00007FF636806AC2]
	(No sym

이번에 찾을 키워드 : 486 / 607 행 혜화동 에이다
이번에 찾을 키워드 : 487 / 607 행 혜화동 플로어칠
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=117.0.5938.92)
Stacktrace:
	GetHandleVerifier [0x00007FF636897892+54818]
	(No symbol) [0x00007FF636806AC2]
	(No symbol) [0x00007FF6366BDA3B]
	(No symbol) [0x00007FF6366FE4FC]
	(No symbol) [0x00007FF6366FE67C]
	(No symbol) [0x00007FF636739627]
	(No symbol) [0x00007FF63671EAEF]
	(No symbol) [0x00007FF6367375A2]
	(No symbol) [0x00007FF63671E883]
	(No symbol) [0x00007FF6366F3691]
	(No symbol) [0x00007FF6366F48D4]
	GetHandleVerifier [0x00007FF636BFB992+3610402]
	GetHandleVerifier [0x00007FF636C51860+3962352]
	GetHandleVerifier [0x00007FF636C49D4F+3930847]
	GetHandleVerifier [0x00007FF636933646+693206]
	(No symbol) [0x00007FF636811628]
	(No symbol) [0x00007FF63680D934]
	(No symbol) [0x00007FF

이번에 찾을 키워드 : 528 / 607 행 혜화동 대성정육식당
이번에 찾을 키워드 : 529 / 607 행 혜화동 비스라운지
이번에 찾을 키워드 : 530 / 607 행 혜화동 케이에스
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=117.0.5938.92)
Stacktrace:
	GetHandleVerifier [0x00007FF636897892+54818]
	(No symbol) [0x00007FF636806AC2]
	(No symbol) [0x00007FF6366BDA3B]
	(No symbol) [0x00007FF6366FE4FC]
	(No symbol) [0x00007FF6366FE67C]
	(No symbol) [0x00007FF636739627]
	(No symbol) [0x00007FF63671EAEF]
	(No symbol) [0x00007FF6367375A2]
	(No symbol) [0x00007FF63671E883]
	(No symbol) [0x00007FF6366F3691]
	(No symbol) [0x00007FF6366F48D4]
	GetHandleVerifier [0x00007FF636BFB992+3610402]
	GetHandleVerifier [0x00007FF636C51860+3962352]
	GetHandleVerifier [0x00007FF636C49D4F+3930847]
	GetHandleVerifier [0x00007FF636933646+693206]
	(No symbol) [0x00007FF636811628]
	(No symbol) [0x000

이번에 찾을 키워드 : 554 / 607 행 혜화동 나래함박
이번에 찾을 키워드 : 555 / 607 행 혜화동 토리아에즈
이번에 찾을 키워드 : 556 / 607 행 혜화동 몬스터헝그리덮밥
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=117.0.5938.92)
Stacktrace:
	GetHandleVerifier [0x00007FF636897892+54818]
	(No symbol) [0x00007FF636806AC2]
	(No symbol) [0x00007FF6366BDA3B]
	(No symbol) [0x00007FF6366FE4FC]
	(No symbol) [0x00007FF6366FE67C]
	(No symbol) [0x00007FF636739627]
	(No symbol) [0x00007FF63671EAEF]
	(No symbol) [0x00007FF6367375A2]
	(No symbol) [0x00007FF63671E883]
	(No symbol) [0x00007FF6366F3691]
	(No symbol) [0x00007FF6366F48D4]
	GetHandleVerifier [0x00007FF636BFB992+3610402]
	GetHandleVerifier [0x00007FF636C51860+3962352]
	GetHandleVerifier [0x00007FF636C49D4F+3930847]
	GetHandleVerifier [0x00007FF636933646+693206]
	(No symbol) [0x00007FF636811628]
	(No symbol) [0x0

이번에 찾을 키워드 : 597 / 607 행 혜화동 쿄코코
이번에 찾을 키워드 : 598 / 607 행 혜화동 의원상
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=117.0.5938.92)
Stacktrace:
	GetHandleVerifier [0x00007FF636897892+54818]
	(No symbol) [0x00007FF636806AC2]
	(No symbol) [0x00007FF6366BDA3B]
	(No symbol) [0x00007FF6366FE4FC]
	(No symbol) [0x00007FF6366FE67C]
	(No symbol) [0x00007FF636739627]
	(No symbol) [0x00007FF63671EAEF]
	(No symbol) [0x00007FF6367375A2]
	(No symbol) [0x00007FF63671E883]
	(No symbol) [0x00007FF6366F3691]
	(No symbol) [0x00007FF6366F48D4]
	GetHandleVerifier [0x00007FF636BFB992+3610402]
	GetHandleVerifier [0x00007FF636C51860+3962352]
	GetHandleVerifier [0x00007FF636C49D4F+3930847]
	GetHandleVerifier [0x00007FF636933646+693206]
	(No symbol) [0x00007FF636811628]
	(No symbol) [0x00007FF63680D934]
	(No symbol) [0x00007FF6

In [19]:
hyehwa.head(10)

,name,dong,lon,lat,naver_keyword,naver_map_url
0,낫컴플리트,혜화동,37.584204,126.998096,혜화동 낫컴플리트,1141273055
1,유환푸드,혜화동,37.586557,127.001484,혜화동 유환푸드,NaN
2,버거파크,혜화동,37.582159,127.001350,혜화동 버거파크,13158227
3,디오엠,혜화동,37.583615,126.998727,혜화동 디오엠,1065126067
4,명륜진사갈비,혜화동,37.585827,126.995108,혜화동 명륜진사갈비,1233726670
5,로젤,혜화동,37.581819,126.999441,혜화동 로젤,1504672949
6,니카이야스미,혜화동,37.582523,126.999590,혜화동 니카이야스미,NaN
7,스시사소우,혜화동,37.582780,126.999179,혜화동 스시사소우,120171872
8,육미안,혜화동,37.583242,126.999434,혜화동 육미안,1718133788
9,워터스트리트,혜화동,37.588531,126.999386,혜화동 워터스트리트,76233511


In [20]:
# 온전한 URL로 변환
hyehwa['naver_map_url'] = "https://m.place.naver.com/restaurant/" + hyehwa['naver_map_url']

In [21]:
# 데이터 확인
hyehwa.head(10)

,name,dong,lon,lat,naver_keyword,naver_map_url
0,낫컴플리트,혜화동,37.584204,126.998096,혜화동 낫컴플리트,https://m.place.naver.com/restaurant/1141273055
1,유환푸드,혜화동,37.586557,127.001484,혜화동 유환푸드,NaN
2,버거파크,혜화동,37.582159,127.001350,혜화동 버거파크,https://m.place.naver.com/restaurant/13158227
3,디오엠,혜화동,37.583615,126.998727,혜화동 디오엠,https://m.place.naver.com/restaurant/1065126067
4,명륜진사갈비,혜화동,37.585827,126.995108,혜화동 명륜진사갈비,https://m.place.naver.com/restaurant/1233726670
5,로젤,혜화동,37.581819,126.999441,혜화동 로젤,https://m.place.naver.com/restaurant/1504672949
6,니카이야스미,혜화동,37.582523,126.999590,혜화동 니카이야스미,NaN
7,스시사소우,혜화동,37.582780,126.999179,혜화동 스시사소우,https://m.place.naver.com/restaurant/120171872
8,육미안,혜화동,37.583242,126.999434,혜화동 육미안,https://m.place.naver.com/restaurant/1718133788
9,워터스트리트,혜화동,37.588531,126.999386,혜화동 워터스트리트,https://m.place.naver.com/restaurant/76233511


In [22]:
# 데이터 저장
hyehwa.to_csv('hyehwa_url.csv', index = False)

# 크롤링

In [3]:
# 데이터 불러오기
df_h = pd.read_csv('Hyehwa_url.csv')

# 위에서부터 쭉 진행한 것이 아니라면 필요한 라이브러리 재실행 필요

In [4]:
# 각 데이터들을 리스트로 

naver_map_name_list = []
naver_map_type_list = []
review_qty_list = []
visitor_review_qty_list = []
blog_review_qty_list = []
naver_map_star_review_stars_list = []
naver_map_star_review_qty_list = []

chromedriver = r'C:/Users/Admin/OneDrive/바탕 화면/Sungkyun University/비교과/DScover 학회(2023)/2023년도 2학기/메인 프로젝트/chromedriver' 

In [7]:
# 웹 드라이버 접속
driver = webdriver.Chrome(chromedriver) 

C:\Users\Admin\AppData\Local\Temp/ipykernel_7712/114379940.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)


In [8]:
# 진행바 표시
from tqdm.notebook import tqdm

In [9]:
for i, url in enumerate(tqdm(df_h['naver_map_url'])):
    
    if pd.isna(url) :
        # 임시로 null 값을 넣음
        
        print(f"{i}행 비어있음")
        
        naver_map_type_list.append(np.nan)
        visitor_review_qty_list.append(np.nan)
        blog_review_qty_list.append(np.nan)
        naver_map_star_review_stars_list.append(np.nan)
        
        i = i+1
        
    else :
        driver.get(url)
        
        time.sleep(2)


        try:
            # 네이버 지도의 유형 분류
            naver_map_type = driver.find_element(By.CLASS_NAME, "DJJvD").text
            
            # 방문자 리뷰 수
            visitor_review_qty = driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm > div.dAsGb > span:nth-child(2) > a > em").text
            
            # 블로그 리뷰 수
            blog_review_qty = driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm > div.dAsGb > span:nth-child(3) > a > em").text
            
            # 별점 점수
            star_review_stars = driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm > div.dAsGb > span.PXMot.LXIwF > em").text
            
            # 별점 평가 수
            # star_review_qty = driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section.no_margin.mdJ86 > div > div > div.Xj_yJ > span:nth-child(2)").text
            # 질문
            
            naver_map_type_list.append(naver_map_type)
            visitor_review_qty_list.append(visitor_review_qty)
            blog_review_qty_list.append(blog_review_qty)
            naver_map_star_review_stars_list.append(star_review_stars)
            # naver_map_star_review_qty_list.append(star_review_qty)

        # 별점이 없는 업체는 오류 발생
        except Exception as e1:
            
            print(f"{i}행 문제가 발생")

            # 임시로 null 값을 넣음       
            naver_map_type_list.append(naver_map_type)
            visitor_review_qty_list.append(visitor_review_qty)
            blog_review_qty_list.append(blog_review_qty)
            naver_map_star_review_stars_list.append(np.nan)
            
            

driver.quit()

  0%|          | 0/608 [00:00<?, ?it/s]

1행 비어있음
4행 문제가 발생
6행 비어있음
7행 문제가 발생


KeyboardInterrupt: 

In [10]:
# 데이터가 잘 들어갔는지 확인
print(naver_map_type_list[0:3])
print(visitor_review_qty_list[0:3])
print(blog_review_qty_list[0:3])
print(naver_map_star_review_stars_list[0:3])

['카페,디저트', nan, '햄버거']
['442', nan, '2,094']
['358', nan, '1,159']
['4.56', nan, '4.51']


In [67]:
# 데이터 정리
df_h['naver_store_type'] = naver_map_type_list  # 네이버 상세페이지에서 크롤링한 업체 유형
df_h['naver_star_point'] = naver_map_star_review_stars_list  # 네이버 상세페이지에서 평가한 별점 평점
df_h['naver_visitor_review_qty'] = visitor_review_qty_list  # 네이버 상세페이지에 나온 방문자 리뷰의 총 개수
df_h['naver_blog_review_qty'] = blog_review_qty_list  # 네이버 상세페이지에 나온 블로그 리뷰의 총 개수

In [100]:
# 방문자 리뷰, 블로그 리뷰 수에 들어있는 숫자 쉼표 제거
for i in range(len(df_h['naver_visitor_review_qty'])):
    df_h['naver_visitor_review_qty'][i] = str(df_h['naver_visitor_review_qty'][i]).replace(',', '')
    
for i in range(len(df_h['naver_blog_review_qty'])):
    df_h['naver_blog_review_qty'][i] = str(df_h['naver_blog_review_qty'][i]).replace(',', '')

C:\Users\Admin\AppData\Local\Temp/ipykernel_2604/561311230.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_h['naver_visitor_review_qty'][i] = str(df_h['naver_visitor_review_qty'][i]).replace(',', '')
C:\Users\Admin\AppData\Local\Temp/ipykernel_2604/561311230.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_h['naver_blog_review_qty'][i] = str(df_h['naver_blog_review_qty'][i]).replace(',', '')


In [106]:
# 별점 평균, 리뷰수 는 numeric으로 변환.
df_h[['naver_star_point', 'naver_visitor_review_qty', 'naver_blog_review_qty']] = df_h[['naver_star_point', 'naver_visitor_review_qty', 'naver_blog_review_qty']].apply(pd.to_numeric, errors = 'coerce')

In [110]:
# 데이터 저장
df_h.to_csv('hyehwa_review.csv', index = False)

# review text 수집

In [1]:
import pandas as pd

In [2]:
# 전처리 완료한 데이터 불러오기
df = pd.read_csv('hyehwa_review.csv')

In [3]:
# 필요한 라이브러리 실행
import pandas as pd 
import numpy as np
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By

import time
import re 
from bs4 import BeautifulSoup 
from tqdm import tqdm 

In [4]:
# 웹드라이버 접속
driver = webdriver.Chrome(r'C:/Users/Admin/OneDrive/바탕 화면/Sungkyun University/비교과/DScover 학회(2023)/2023년도 2학기/메인 프로젝트/chromedriver')

C:\Users\Admin\AppData\Local\Temp/ipykernel_17852/2170489192.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:/Users/Admin/OneDrive/바탕 화면/Sungkyun University/비교과/DScover 학회(2023)/2023년도 2학기/메인 프로젝트/chromedriver')


In [5]:
# 빈 리스트 생성
review_list = []
review_num_list = []

In [6]:
for i in range(len(df)): 

    print('======================================================') 
    print(str(i)+'번째 식당') 
    
    if pd.isna(df['naver_map_url'][i]) :
        print(str(i)+'번째 행은 비어있음') 
        review_list.append(np.nan)
        review_num_list.append(np.nan)
        i = i+1
    
    else:
        # 식당 리뷰 개별 url 접속
        driver.get(df['naver_map_url'][i]+'/review/visitor') 
        time.sleep(2) 

        # 더보기 버튼 다 누를 것
        # 더보기 버튼은 10개 마다 나옴
        while True: 
            try: 
                time.sleep(1) 
                driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END) 
                time.sleep(3) 

                pluskey = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section.k5tcc > div.lfH3O > a') 
                pluskey.send_keys(Keys.ENTER)
                time.sleep(3) 
                
                driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END) 
                time.sleep(1) 

            except NoSuchElementException: 
                print('-더보기 버튼 모두 클릭 완료-') 
                break 


        # 파싱
        html = driver.page_source 
        soup = BeautifulSoup(html, 'lxml') 
        time.sleep(1) 

        # 식당 구분 
        restaurant_name = df['naver_keyword'][i]
        print('식당 이름 : '+restaurant_name) 

        # 특정 식당에 대한 리뷰 수집
        try: 
            one_review = soup.find_all('span', attrs = {'class' : 'zPfVt'})
            review_num = len(one_review) # 특정 식당의 리뷰 총 개수 
            print('리뷰 총 개수 : '+str(review_num)) 

            # review 내용
            try: 
                review_content_list = []
                for i in range(review_num):
                    review_content = one_review[i].text
                    review_content_list.append(review_content)
                
            except: # 리뷰가 없다면
                review_content = "" 
                
            review_list.append(review_content_list)
            print('마지막 리뷰 내용 : '+review_content) 


        # 리뷰가 없는 경우        
        except NoSuchElementException: 
            none_review = "네이버 리뷰 없음" 
            print(none_review)
            review_num = 0 
        
        review_num_list.append(review_num)

print('\n')

0번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 낫컴플리트
리뷰 총 개수 : 166
마지막 리뷰 내용 : 괜찮아요
1번째 식당
1번째 행은 비어있음
2번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 버거파크
리뷰 총 개수 : 883
마지막 리뷰 내용 : 그냥 수제버거집.. 파인애플을 넣어야 맛있음
3번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 디오엠
리뷰 총 개수 : 38
마지막 리뷰 내용 : 시샤 좋습니다..
4번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 명륜진사갈비
리뷰 총 개수 : 158
마지막 리뷰 내용 : 좋아요
5번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 로젤
리뷰 총 개수 : 77
마지막 리뷰 내용 : 사장님짱 맛도 좋아요
6번째 식당
6번째 행은 비어있음
7번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 스시사소우
리뷰 총 개수 : 390
마지막 리뷰 내용 : 점심셋트 저렴하고 좋음
8번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 육미안
리뷰 총 개수 : 556
마지막 리뷰 내용 : 모처럼 외식 좋았어요
9번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 워터스트리트
리뷰 총 개수 : 29
마지막 리뷰 내용 : 분위기 예뻐요
10번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 째즈&라멘
리뷰 총 개수 : 8
마지막 리뷰 내용 : 내가 이런 운동 할 수 있을까 싶었는데,
자세하고 친절하게 알려주시니까 되네요?
하나씩 배우는 재미와 성취감이 있습니다~~
11번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 떼루와
리뷰 총 개수 : 81
마지막 리뷰 내용 : 그냥 저렴
맛은 무난
테이크아웃 전용매장
12번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 스시마시따
리뷰 총 개수 : 277
마지막 리뷰 내용 : 굿
13번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름

-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 히메카츠
리뷰 총 개수 : 82
마지막 리뷰 내용 : 넘넘 맛있고 가격도 합리적이에염
66번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 히든스시
리뷰 총 개수 : 51
마지막 리뷰 내용 : 완전 강추해요 특히 새우초밥 굿!
67번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 고쿠텐
리뷰 총 개수 : 281
마지막 리뷰 내용 : 텐동퀄리티가대박! 단골확정이요 넘맛있어요
68번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 춘참치포차
리뷰 총 개수 : 31
마지막 리뷰 내용 : 가성비가 좋아요~참치초밥이 황칠향인지 은은한 향이나서 맛있어요. 추가 주문을 했으나 부엌에서 신경을 안쓰셔서 개선이 필요하다고 생각합니다.
69번째 식당
69번째 행은 비어있음
70번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 포차야
리뷰 총 개수 : 3
마지막 리뷰 내용 : 굿
71번째 식당
71번째 행은 비어있음
72번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 코코버블티
리뷰 총 개수 : 59
마지막 리뷰 내용 : Good
73번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 본우리집밥재능교육본사점
리뷰 총 개수 : 10
마지막 리뷰 내용 : 가성비 좋음
74번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 쇼타돈부리
리뷰 총 개수 : 129
마지막 리뷰 내용 : 육회동이 신메뉴로 생겼다는 소식을 듣고갔는데 너무 맛없었다.. 육회동 남기긴 처음이야.. 쇼타에서는 기본 메뉴만 먹는걸로. 육회만 올린다고 육회동이 아님을 절실히 깨달음..
75번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 세아
리뷰 총 개수 : 214
마지막 리뷰 내용 : 갓성비
76번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 옥이모
리뷰 총 개수 : 424
마지막 리뷰 내용 : 굿
77번째 식당
-더보기 버튼 모두 클릭 

-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 리춘시장대학로
리뷰 총 개수 : 10
마지막 리뷰 내용 : 사막깐풍기. 쏘쏘. 여러 안주를 먹었으나 가성비 높지 않음. 뚝배기마파두부 추천.
124번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 오름
리뷰 총 개수 : 10
마지막 리뷰 내용 : 동기 모임하느라고 여섯명이 갔는데 제주 컨셉으로 고기도 맛있고  하이볼도 맛있고 꿀라봉도 맛있어요 딱새우랑 전복 나오는 오름 한차롱이랑 오름 
몬딱세트 먹고 생갈비3 목살2 삼겹살2 추가로 더 시켜먹고 (각자 다 맛있다고 생각하는 메뉴를 더 시키다보니) 마무리로 라면 두개 시켰는데 새우도 들
어있고 조개도 들어있어서 비주얼 좋고 마무리로 얼큰하니 좋았어요 재방문 의사 있습니다^^
125번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 혜화달손만두,만두전골
리뷰 총 개수 : 10
마지막 리뷰 내용 : 브레이크 타임 없어서 늦은 점심 먹었어요👍🏼
126번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 서화커피
리뷰 총 개수 : 10
마지막 리뷰 내용 : 고즈넉하고 예쁜 한옥 카페입니다. 내부 공간은 ㄱ자이고, 공간이 넓진 않으며 4인 독실이 따로 있어용. 동양화 작품이 포인트로 인테리어가 되어있는
데, 고풍스럽고 이뻐요! 주말 저녁시간대에 약간의 웨이팅이 있네요. 저희가 갔을땐 여자사장님 한분만 일하셨는데, 음료제조에 홀정리 웨이팅관리까
지 하셔서 바쁘실텐데, 환한 미소로 응대 잘 해주셔서 감사했습니다!
127번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 혜화시장
리뷰 총 개수 : 10
마지막 리뷰 내용 : 매운사태찜이랑 한우튀김은 필수 ㅠ 진짜 짱맛탱이에요
128번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 곤조
리뷰 총 개수 : 10
마지막 리뷰 내용 : 오 여기 생긴지 얼마 안된 곳인지.. 아무튼 음식도 넘 맛있고 도란도란 얘기하며 술 먹기 너무 좋았음. 
129번째 식당
-더보기 버튼

-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 남대문황금족발
리뷰 총 개수 : 61
마지막 리뷰 내용 : 걍 뭐 그럭저럭 평범한 ㄱㅊㄱㅊ
172번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 티랩제이
리뷰 총 개수 : 121
마지막 리뷰 내용 : 친절한 사장님이 해주신 친절한 설명  덕분에
힐링도 되고 차에 대한 상식도 풍부해지고...
정말 맛있는 차 마시면서 행복한 시간 보내고 왔습니다
173번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 파스타마켓
리뷰 총 개수 : 734
마지막 리뷰 내용 : 맛있어요
174번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 봄까스
리뷰 총 개수 : 50
마지막 리뷰 내용 : 굿
175번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 봉심
리뷰 총 개수 : 289
마지막 리뷰 내용 : 음식 다 맛있어요!! 그리고 안에 있는 시바견이 진짜 귀엽..
176번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 맛밥김밥
리뷰 총 개수 : 615
마지막 리뷰 내용 : 혜화 맛집 인증 ❤
177번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 도광
리뷰 총 개수 : 0
마지막 리뷰 내용 : 혜화 맛집 인증 ❤
178번째 식당
178번째 행은 비어있음
179번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 재팔이네닭발
리뷰 총 개수 : 78
마지막 리뷰 내용 : 굿
180번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 카페혜화로
리뷰 총 개수 : 404
마지막 리뷰 내용 : 좋아요
181번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 대동원회시장
리뷰 총 개수 : 186
마지막 리뷰 내용 : 굿
182번째 식당
182번째 행은 비어있음
183번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 04판
리뷰 총 개수 : 255
마지막 리뷰 내용 : 굿
184번째 식당
-더보기 버튼 모두 클릭

-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 빌런치팬케이크
리뷰 총 개수 : 52
마지막 리뷰 내용 : 굳
238번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 창경궁초밥
리뷰 총 개수 : 195
마지막 리뷰 내용 : 굳
239번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 아마스빈
리뷰 총 개수 : 253
마지막 리뷰 내용 : 굿
240번째 식당
240번째 행은 비어있음
241번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 메르시
리뷰 총 개수 : 29
마지막 리뷰 내용 : 굿
242번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 혜화도담
리뷰 총 개수 : 904
마지막 리뷰 내용 : 연극보기전에 김치찜이 먹고싶어서 예약한거였는데 너무 맛있게 먹었어요~ 집밥먹는 기분이였습니다 ㅎㅎ 다음엔 갈비찜 먹으러 또 갈게요!
243번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 뜸들이다
리뷰 총 개수 : 29
마지막 리뷰 내용 : 굳
244번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 명쭈3대학로
리뷰 총 개수 : 53
마지막 리뷰 내용 : 맛있어요
245번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 육쌈냉면
리뷰 총 개수 : 186
마지막 리뷰 내용 : 격하게 맛있어요
246번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 굿네스카페
리뷰 총 개수 : 9
마지막 리뷰 내용 : 커피가 맛있어요 
247번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 마피
리뷰 총 개수 : 89
마지막 리뷰 내용 : 최고입니다!!!!!!
248번째 식당
248번째 행은 비어있음
249번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 베네딕션
리뷰 총 개수 : 74
마지막 리뷰 내용 : 굿
250번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 통큰갈비
리뷰 총 개수 : 167
마지막 리뷰 내용 : 갈비
25

...

-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 돈스쿨
리뷰 총 개수 : 10
마지막 리뷰 내용 : 삼겹살에 김치, 버섯, 마늘, 콩나물, 양파 구워먹으면 맛있고, 서비스 된장찌개가 정말 맛있어요. 가성비 짱!
362번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 디커피
리뷰 총 개수 : 11
마지막 리뷰 내용 : 성균관 대학교로 들어가는 골목 입구에 있는 더치커피 전문점이에요. 커피를 4종류 정도 구비해두고 계셔요. 
363번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 일미식당
리뷰 총 개수 : 20
마지막 리뷰 내용 : 친절합니다
364번째 식당
364번째 행은 비어있음
365번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 릴렉스인다운타운
리뷰 총 개수 : 51
마지막 리뷰 내용 : 밀크티 완전 맛잇어요 :)
366번째 식당
366번째 행은 비어있음
367번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 노가네
리뷰 총 개수 : 0
마지막 리뷰 내용 : 밀크티 완전 맛잇어요 :)
368번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 참숯불갈비살
리뷰 총 개수 : 81
마지막 리뷰 내용 : 친절해요
369번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 9번지해장국
리뷰 총 개수 : 32
마지막 리뷰 내용 : 그럭저럭
370번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 미술관
리뷰 총 개수 : 46
마지막 리뷰 내용 : 추억의 미술관
 
다들 건강하고 행복했으면
371번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 명륜동식당
리뷰 총 개수 : 103
마지막 리뷰 내용 : 친절하세요^^
372번째 식당
372번째 행은 비어있음
373번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 페르시아궁전
리뷰 총 개수 : 1663
마지막 리뷰 내용 : 최고예오~!!
374번째 식당
374번째 행은 비어있음
375번째 식당
3

식당 이름 : 혜화동 이유
리뷰 총 개수 : 281
마지막 리뷰 내용 : 테이블이 몇개없는 작은식당인데
정성이 느껴지는 집밥 느낌이었어요
생선구이 오랜만에 먹었는데 양 많고 맛있었어요
424번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 카페자리
리뷰 총 개수 : 144
마지막 리뷰 내용 : 진짜 비싸요
425번째 식당
425번째 행은 비어있음
426번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 삼포
리뷰 총 개수 : 6
마지막 리뷰 내용 : 감사합니다
맛나요

427번째 식당
427번째 행은 비어있음
428번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 명륜감자탕
리뷰 총 개수 : 62
마지막 리뷰 내용 : 좋아요
429번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 케이디디컴퍼니
리뷰 총 개수 : 1979
마지막 리뷰 내용 : 마제소바 처음먹어봤는데 맛있네요 
430번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 차니오뎅
리뷰 총 개수 : 312
마지막 리뷰 내용 : 대학로에 연극 보러 갔다가 우연히 발견하고 식사하러 들어간 식당인데 보들보들하면서 쫀쫀한 어묵이 너무 맛있었어요!!!
431번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 주항아리
리뷰 총 개수 : 29
마지막 리뷰 내용 : #혜화 #대학로 #유자막걸리 존맛
해물라면도 존맛
432번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 만연채
리뷰 총 개수 : 52
마지막 리뷰 내용 : 사장님의 여유로움이 음료에 담겨서 너무 좋습니다 :)
433번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 은식당
리뷰 총 개수 : 212
마지막 리뷰 내용 : 맛있어요
434번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 학림커피
리뷰 총 개수 : 378
마지막 리뷰 내용 : 비싸요
435번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 오늘도한잔
리뷰 총 개

식당 이름 : 혜화동 바오쯔
리뷰 총 개수 : 241
마지막 리뷰 내용 : 맛있었어요
485번째 식당
485번째 행은 비어있음
486번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 에이다
리뷰 총 개수 : 0
마지막 리뷰 내용 : 맛있었어요
487번째 식당
487번째 행은 비어있음
488번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 마로화적
리뷰 총 개수 : 78
마지막 리뷰 내용 : 분위기 좋고 음식이 맛있는 한식집이에요. 인테리어가 한옥이라, 더 멋스러워요. 다양한 한국식 주류도 취급하는데 음식하고 잘 어우러집니다.
489번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 환식당
리뷰 총 개수 : 84
마지막 리뷰 내용 : 파스타 맛있는 숨은 맛집이예요
490번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 이자카야진
리뷰 총 개수 : 121
마지막 리뷰 내용 : 서비스 굿 꼬치맛도 가성비 좋아요
491번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 핫브레드혜화역점
리뷰 총 개수 : 759
마지막 리뷰 내용 : 처음 이곳을 이용해봤는데 너무 맛있는 빵을 구매했어요.
492번째 식당
492번째 행은 비어있음
493번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 온프
리뷰 총 개수 : 180
마지막 리뷰 내용 : 맛있어용용~
494번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 해피니스
리뷰 총 개수 : 391
마지막 리뷰 내용 : 맛있게 먹었네용! 
495번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 달콤한잔
리뷰 총 개수 : 30
마지막 리뷰 내용 : 맛있어요
496번째 식당
496번째 행은 비어있음
497번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 카페선선혜화
리뷰 총 개수 : 108
마지막 리뷰 내용 : 따스한 공간~~
498번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 메리그린
리뷰 총 개

-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 마녀주방
리뷰 총 개수 : 1100
마지막 리뷰 내용 : 여자친구가 너무 좋아하네요 감사합니다 잘먹었습니다!
549번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 커피빈성균관대법학관점
리뷰 총 개수 : 3
마지막 리뷰 내용 : 커피와 음료를 주문했어요
550번째 식당
550번째 행은 비어있음
551번째 식당
551번째 행은 비어있음
552번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 커피빈혜화역점
리뷰 총 개수 : 552
마지막 리뷰 내용 : 친절해요 친절해요
553번째 식당
553번째 행은 비어있음
554번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 나래함박
리뷰 총 개수 : 213
마지막 리뷰 내용 : ㅎ
555번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 토리아에즈
리뷰 총 개수 : 64
마지막 리뷰 내용 : 완전 존맛탱 닭목살 관자 존맛탱
556번째 식당
556번째 행은 비어있음
557번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 우리의밤
리뷰 총 개수 : 57
마지막 리뷰 내용 : 분위기가 정말 좋았습니다 :) 칵테일 작명 센스도 최고였어요ㅎㅎ
558번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 백소정
리뷰 총 개수 : 1928
마지막 리뷰 내용 : 재료가 신선하고 맛있어요 🥰
559번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 에스프레소그자체
리뷰 총 개수 : 57
마지막 리뷰 내용 : 아이스아메리카노 쓴맛이 너무 적어서 아쉬웠어요ㅠ
560번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 라온섬
리뷰 총 개수 : 33
마지막 리뷰 내용 : 좋아요
561번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 왕가탕후루
리뷰 총 개수 : 9
마지막 리뷰 내용 : 달아요~
562번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 발리밥
리뷰 총 

-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 삼삼뼈국
리뷰 총 개수 : 10
마지막 리뷰 내용 : 오늘도 넘넘넘넘 잘 먹었습니당!!! 토마토 바질 뼈찜 먹고 후식으로 파스타 진짜 필수... 넘 맛있어요!!!! >_< 고기가 오또케 이로케 부드러울 수 있죵... 대
학로 맛집으로 인정합니당!!!ㅎㅎ
607번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 혜화동 인쌩맥주혜화점
리뷰 총 개수 : 6
마지막 리뷰 내용 : 새로생겨서 깨끗!!!!




In [9]:
# 데이터 정리
df['naver_visitor_review_text'] = review_list
df['naver_visito_review_num'] = review_num_list

In [10]:
# 데이터 확인
df

,name,dong,lon,lat,naver_keyword,naver_map_url,naver_store_type,naver_star_point,naver_visitor_review_qty,naver_blog_review_qty,naver_visitor_review_text,naver_visito_review_num
0,낫컴플리트,혜화동,37.584204,126.998096,혜화동 낫컴플리트,https://m.place.naver.com/restaurant/1141273055,"카페,디저트",4.56,441.0,359.0,[진짜 크로플 맛집이에요 🧇🍨 제가 제일 좋아하는 카페 ! 분위기도 정말 좋고 음료...,166.0
1,유환푸드,혜화동,37.586557,127.001484,혜화동 유환푸드,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,버거파크,혜화동,37.582159,127.001350,혜화동 버거파크,https://m.place.naver.com/restaurant/13158227,햄버거,4.51,2088.0,1161.0,[대학로에서 가장맛있는 버거집이 아닐까 ? \n세트 할인 없는건 아쉽지만 .. 맛있...,883.0
3,디오엠,혜화동,37.583615,126.998727,혜화동 디오엠,https://m.place.naver.com/restaurant/1065126067,바(BAR),4.95,45.0,8.0,"[제대로 된 서울 시샤 맛집 찾으실 때 꼭 가보세요, 저랑 여자친구는 그래도 나름 ...",38.0
4,명륜진사갈비,혜화동,37.585827,126.995108,혜화동 명륜진사갈비,https://m.place.naver.com/restaurant/1233726670,"육류,고기요리",NaN,120.0,8.0,[명륜진사갈비가 새로와졌네요. 우와~~ 분위기가 멋지니깐 더 멋있는것 같아요. 서울...,158.0
...,...,...,...,...,...,...,...,...,...,...,...,...
603,하얀구르미,혜화동,37.584636,127.000162,혜화동 하얀구르미,NaN,NaN,NaN,NaN,NaN,NaN,NaN
604,에디션엠,혜화동,37.582128,127.000830,혜화동 에디션엠,NaN,NaN,NaN,NaN,NaN,NaN,NaN
605,엠아이피,혜화동,37.588980,126.994963,혜화동 엠아이피,NaN,NaN,NaN,NaN,NaN,NaN,NaN
606,삼삼뼈국,혜화동,37.582797,127.001183,혜화동 삼삼뼈국,https://m.place.naver.com/restaurant/1582796565,한식,NaN,94.0,241.0,[제 맘 속일 등이에요 깻잎 들게 안 들어 가고 한우사골육수를 진하게 우려낸 감자탕...,10.0


In [30]:
# 리뷰텍스트가 잘 입력되었는지 재확인
df['naver_visitor_review_text'][2]

'[\'대학로에서 가장맛있는 버거집이 아닐까 ? \\n세트 할인 없는건 아쉽지만 .. 맛있고, 비싼만큼 재료가 가득 들어가서.. 가끔씩 버거 땡길때 여기루 온다.. 매장도 분위기 좋다.. 햄버거집인데. .약간 퓨전 한식 인테리어 같당 ㅋ \', \'식물이 많아서 좋아요! 안에 인테리어도 예쁘고 야외자리 실내자리도 넉넉해서 대화하기도 좋아요-!\', \'혜화역 수제버거 맛집 분위기도 좋고 햄버거도 상당히 맛있습니다 🍔🥤👍 인스타 sung.hee_\', \'치킨패티 오져요 짠 맛 아니고 건강한 맛!!!\', \'수제버거 먹어본 중 최고 맛있는 것 같아요 매장내에 음료 자유롭게 리필하는 것도 좋아요\', \'부드러운 패티와 고소한 치즈, 잘 구워진 번이 맛있는 곳이에요\', \'일반 패스트푸드 햄버거가 아닌 수제버거가 먹고 싶던 차에~~ 맛있네!! 치즈버거 세트와 아보카도 치즈버거 세트 나눠 먹기~ 개인적으로 치즈버거가 \\n더 맛있네~!\', \'재료 하나하나가 신선한 느낌\\n감튀도 따뜻하고 기름기 쏙 빠진게 참 좋았어요\', \'감튀는 크링클컷이 진리입니다. 오늘 버거는 따끈하고 맛이 좋았습니다. \', \'오랜만에 혼밥이라 뭘 먹을까 고민 많이 했었는데요, 돌고돌아 버거파크 왔습니다. 바삭하고 맛있는 버거 맛을 잊지 못해서요...ㅠㅠ 자주 먹어도 계속 \\n생각나는 맛입니다!!\', \'치즈버거 더블 세트로 먹음 14,800원 가성비는 떨어지지만,  고기육즙이 나오고, 좀 짜서 맛있음.  양상추가 조금이라 아쉽, 감튀맛은 쏘쏘. 수제버거 \\n좋아한다면 추천~!\', \'햄버거 패티도 맛있고 기본적인 햄버거 맛에 충실해서 질리지 않고 담백하고 맛있게 먹었습니다 :) 그리고 감자튀김이 크링클컷이어서 그런지 더 바삭\\n해서 좋았어요🙂 너무 맛있었어요!\', \'요즘 수제버거에 꽂혔는데 여기 맛있다고 해서 혼밥하러 왔습니다. 생각보다 매장이 넓고 직원분들도 친절하세요.\\n햄버거도 먹기 쉽게 잘 배치해주시고, 번이 폭신하니 너무 맛있었어요. 다른 재료들도 신선하

In [14]:
# 데이터 저장
df.to_csv('hyehwa_review.csv', index = False)